In [ ]:
import datetime as dt
import pandas as pd
import requests
from prefect.blocks.system import Secret
from sqlalchemy import create_engine, select
from sqlalchemy.orm import Session
from mcpdb.tables import Provider, ProviderAsset

In [ ]:
postgresql_password: str = (await Secret.load("postgresql-password")).get()
host = "db-postgresql-lon1-65351-do-user-18535103-0.m.db.ondigitalocean.com"
port = 25060
database = "defaultdb"
user = "doadmin"
url = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(
    user=user,
    password=postgresql_password,
    host=host,
    port=port,
    database=database,
)
engine = create_engine(url)

In [ ]:
categories: list[str] = []
with Session(engine) as session:
    # Get the coin desk provider.
    stmt = select(Provider).where(Provider.name == "CoinDesk")
    provider = session.execute(stmt).scalar_one_or_none()

    # Get all categories from provider assets.
    if provider:
        stmt = select(ProviderAsset).where(ProviderAsset.provider_id == provider.id)
        provider_assets = session.execute(stmt).scalars().all()
        for provider_asset in provider_assets:
            if provider_asset.asset_code:
                categories.append(provider_asset.asset_code)

categories

In [ ]:
host = "https://data-api.coindesk.com"
uri = "/news/v1/article/list"
params = {
    "lang": "EN",
    "limit": 100,
    "to_ts": (dt.datetime.now() - dt.timedelta(hours=2)).timestamp(),
    "categories": ",".join(categories),
    "source_ids" : ",".join([provider.name.lower()])
}
response = requests.get(f"{host}{uri}", params=params)
if response.status_code != 200:
    raise Exception(f"Failed to fetch data from Coindesk API: {response.status_code}")
data = response.json()["Data"]
data

In [ ]:
df = pd.DataFrame(data)
for col in df.columns:
    print(f"Column: {col}, Type: {df[col].dtype}")